<a href="https://colab.research.google.com/github/mkbahk/AmazonBraket/blob/main/QuantumApplicationAlgorithm_VQA_QAOA(GoogleORTool)_mkbahk_20260203.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install ortools

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.8/29.8 MB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.5/323.5 kB 14.3 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.5
    Uninstalling protobuf-5.29.5:
      Successfully uninstalled protobuf-5.29.5
  Attempting uninstall: absl-py
    Found existing installation: absl-py 1.4.0
    Uninstalling absl-py-1.4.0:
      Successfully uninstalled absl-py-1.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.19.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<6.0.0dev,>=3.20.3, but you have protobuf 6.33.5 which is incompatible.
google-ai-generativelanguage 0.6.15 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.

In [2]:
import numpy as np
from ortools.constraint_solver import pywrapcp
from ortools.constraint_solver import routing_enums_pb2

In [3]:
#
# 대한민국의 모든 시도청을 가장 짦은 경로로 여행하고 싶다.
#
# 17개 도시 목록 (인덱스 순서), 거리단위는 Km
# 0:서울, 1:인천, 2:경기(수원), 3:강원(춘천), 4:충북(청주), 5:충남(홍성), 6:세종, 7:대전,
# 8:전북(전주), 9:전남(무안), 10:경북(안동), 11:경남(창원), 12:제주, 13:부산, 14:울산, 15:대구, 16:광주

distance_matrix = np.array([
    [  0,  32,  35, 105, 135, 130, 132, 160, 210, 340, 235, 350, 450, 390, 375, 285, 290],   # 0: 서울
    [ 32,   0,  42, 135, 145, 105, 140, 165, 205, 330, 255, 355, 445, 400, 385, 300, 285],   # 1: 인천
    [ 35,  42,   0, 120, 105,  95, 105, 130, 175, 305, 215, 320, 415, 365, 350, 265, 260],    # 2: 경기
    [105, 135, 120,   0, 170, 225, 195, 215, 305, 430, 230, 390, 540, 415, 370, 290, 385],  # 3: 강원
    [135, 145, 105, 170,   0,  90,  35,  45, 110, 245, 130, 240, 360, 285, 270, 180, 195],    # 4: 충북
    [130, 105,  95, 225,  90,   0,  75,  90, 105, 230, 225, 285, 345, 335, 330, 245, 185],     # 5: 충남
    [132, 140, 105, 195,  35,  75,   0,  25,  95, 230, 155, 245, 345, 290, 280, 190, 180],     # 6: 세종
    [160, 165, 130, 215,  45,  90,  25,   0,  85, 215, 150, 230, 330, 265, 255, 165, 165],     # 7: 대전
    [210, 205, 175, 305, 110, 105,  95,  85,   0, 135, 215, 205, 255, 260, 265, 195,  90],    # 8: 전북
    [340, 330, 305, 430, 245, 230, 230, 215, 135,   0, 345, 260, 180, 305, 330, 290,  65],   # 9: 전남
    [235, 255, 215, 230, 130, 225, 155, 150, 215, 345,   0, 185, 435, 165, 145, 105, 280],  # 10: 경북
    [350, 355, 320, 390, 240, 285, 245, 230, 205, 260, 185,   0, 280,  45,  70, 100, 210],    # 11: 경남
    [450, 445, 415, 540, 360, 345, 345, 330, 255, 180, 435, 280,   0, 310, 340, 380, 240],  # 12: 제주
    [390, 400, 365, 415, 285, 335, 290, 265, 260, 305, 165,  45, 310,   0,  55, 115, 255],    # 13: 부산
    [375, 385, 350, 370, 270, 330, 280, 255, 265, 330, 145,  70, 340,  55,   0,  95, 280],     # 14: 울산
    [285, 300, 265, 290, 180, 245, 190, 165, 195, 290, 105, 100, 380, 115,  95,   0, 205],   # 15: 대구
    [290, 285, 260, 385, 195, 185, 180, 165,  90,  65, 280, 210, 240, 255, 280, 205,   0]     # 16: 광주
])

print(f"Matrix Shape: {distance_matrix.shape}")

Matrix Shape: (17, 17)


In [4]:
def create_data_model():
    """문제 데이터를 정의합니다."""
    data = {}
    # 기존에 정의된 17개 도시의 distance_matrix 사용
    global distance_matrix # 전역 변수 distance_matrix를 사용
    data["distance_matrix"] = distance_matrix.tolist() # numpy array를 list로 변환
    data["num_vehicles"] = 1 # 외판원은 1명
    data["depot"] = 0        # 시작점 (0번 도시)

    # solve_tsp 함수에서 `locations` 변수가 필요하지만 더 이상 사용되지 않으므로, 더미 값을 반환합니다.
    dummy_locations = None
    return data, dummy_locations

In [5]:
def solve_tsp():
    # 1. 데이터 준비
    data, locations = create_data_model()

    # 2. Routing Index Manager 및 Model 설정
    manager = pywrapcp.RoutingIndexManager(len(data["distance_matrix"]),
                                           data["num_vehicles"], data["depot"])
    routing = pywrapcp.RoutingModel(manager)

    # 3. 거리 계산 콜백 함수 등록
    def distance_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data["distance_matrix"][from_node][to_node]
    ###def

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # 4. 여행 비용(거리) 설정
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # 5. 검색 파라미터 설정 (Heuristic 사용)
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
    )

    # 6. 문제 해결
    solution = routing.SolveWithParameters(search_parameters)

    # 7. 결과 출력
    if solution:
        print_solution(manager, routing, solution)
    ###if
###def

In [6]:
def print_solution(manager, routing, solution):
    print(f"최적 경로 탐색 완료!")
    index = routing.Start(0)
    plan_output = "경로: "
    route_distance = 0
    while not routing.IsEnd(index):
        plan_output += f"{manager.IndexToNode(index)} -> "
        previous_index = index
        index = solution.Value(routing.NextVar(index))
        route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
    ###while

    plan_output += f"{manager.IndexToNode(index)}\n"
    print(plan_output)
    print(f"총 이동 거리: {route_distance} units")
###def

In [7]:
if __name__ == "__main__":
    solve_tsp()
###if

최적 경로 탐색 완료!
경로: 0 -> 1 -> 2 -> 5 -> 4 -> 6 -> 7 -> 8 -> 16 -> 9 -> 12 -> 11 -> 13 -> 14 -> 15 -> 10 -> 3 -> 0

총 이동 거리: 1654 units
